In [3]:
import gurobipy as GRB
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from gurobipy import GRB, quicksum

def normalize_weights(weights, tolerance=1e-9):
    # Check if weights already sum to approximately 1
    total = np.sum(weights)
    if abs(total - 1.0) < tolerance:
        return weights
        
    # Normalize weights
    normalized_weights = weights / total
    
    # Ensure exact sum to 1 by adjusting the largest weight
    sum_diff = 1.0 - np.sum(normalized_weights)
    if abs(sum_diff) > 0:
        max_idx = np.argmax(normalized_weights)
        normalized_weights[max_idx] += sum_diff
        
    # Verify the sum is now exactly 1.0
    assert abs(np.sum(normalized_weights) - 1.0) < tolerance
    
    return normalized_weights


def CVaR_optimization(bond_data, scenario_returns, previous_weights, alpha=0.95):
    S, N = scenario_returns.shape
    print("prev weights: ", previous_weights)
    # Create the model
    model = gp.Model("CVaR_Bond_Optimization")

    model.setParam('OutputFlag', 0)

    # Decision variables
    w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  # weights for bonds
    x = model.addVars(N, vtype=GRB.BINARY, name="x")  # binary selection of bonds
    VaR = model.addVar(vtype=GRB.CONTINUOUS, name="VaR")  # Value at Risk variable
    z = model.addVars(S, vtype=GRB.CONTINUOUS, lb=0, name="z")  # Auxiliary for CVaR
    
    # Objective function: Minimize CVaR risk (USING S)
    model.setObjective(
        VaR + (1 / (S * (1 - alpha))) * gp.quicksum(z[s] for s in range(S)),
        GRB.MINIMIZE
    )
    
    # Constrain the total weight
    model.addConstr(gp.quicksum(w[i] for i in range(N)) >= 0.99 , "SumToOneLower")
    model.addConstr(gp.quicksum(w[i] for i in range(N)) <= 1, "SumToOneUpper")

    # Extract expected returns from the DataFrame
    expected_returns = bond_data["ExpectedReturn"].values  # Assuming "ExpectedReturns" is a column in the DataFrame

    # Coupling weights and binary variables: w[i] = 0 if x[i] = 0
    for i in range(N):
        model.addConstr(w[i] <= x[i], f"WeightCoupling_{i}")
    
    #VaR constraint
    
    model.addConstr(VaR >= gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N) for s in range(S)) / S, "VaR_calculation")

    #CVaR constraint
    
    for s in range(S):  # Iterate through the scenarios
        model.addConstr(z[s] >= gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N)) - VaR, f"cvar_constraint_{s}")


    # deviation from benchmark constraint
    
    weights = np.array([1/N] * N)  # Creates array of 75 elements each with 1/75
    normalized_weights = normalize_weights(weights)
    BenchmarkWeight = normalized_weights[0]  # Since all weights are equal, can take any element
    
    deviation_limit = 0.5  # deviation limit is inputted from user
    for i in range(N):
        model.addConstr(w[i] >= (BenchmarkWeight - deviation_limit*BenchmarkWeight), f"LowerBound_{i}")
        model.addConstr(w[i] <= (BenchmarkWeight + deviation_limit*BenchmarkWeight), f"UpperBound_{i}")

    # sum up to 1 constraint
    e2 = 1e-5
    model.addConstr(gp.quicksum(w[i] for i in range(N)) >= 1 - e2 , "SumToOneLower")
    model.addConstr(gp.quicksum(w[i] for i in range(N)) <= 1 + e2, "SumToOneUpper")
    
    # Spread Constraints
    f = model.addVar(name="f_oas_dev") 
    oas_dev = gp.quicksum((w[i] - BenchmarkWeight)*bond_data.iloc[i]['OAS'] for i in range(N))
    spread_dev = 0.3 # spread_dev is inputted from user 
    model.addConstr(f >= oas_dev, "abs_oas_1")
    model.addConstr(f >= -oas_dev, "abs_oas_2")
    model.addConstr(f <= spread_dev, "oas_deviation_bound")
    
    # Liquidity Constraint
    mean_Liquidity = gp.quicksum(bond_data.iloc[i]['LiquidityScore'] for i in range(N)) / N  
    MinLiquidity = mean_Liquidity  # this value is a placeholder, should be inputted by the user
    model.addConstr(gp.quicksum(bond_data.iloc[i]['LiquidityScore'] * w[i] for i in range(N)) >= MinLiquidity, "MinLiquidity")
    
    # Ratings Constraint
    investment_grade_ratings = ['AAA', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-', 'BBB+', 'BBB', 'BBB-']
    bond_data['is_investment_grade'] = bond_data['Rating'].apply(lambda x: 1 if x in investment_grade_ratings else 0)
    investment_grade_percentage = 0.3   # should be inputted by the user
    investment_grade_weight = gp.quicksum(bond_data.iloc[i]['is_investment_grade'] * w[i] for i in range(N))
    model.addConstr(investment_grade_weight >= investment_grade_percentage * gp.quicksum(w[i] for i in range(N)), "InvestmentGradeConstraint")
    
    # Turnover Constraint
    max_turnover = 0.5  # should be inputted by the user
    # Variables for turnover calculation

    if previous_weights is not None:
        # Variables for turnover calculation
        pos_change = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="pos_change")
        neg_change = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="neg_change")
        
        # Decompose the weight change into positive and negative parts
        for i in range(N):
            # Use previous_weights[i] directly as a constant
            model.addConstr(pos_change[i] >= w[i] - previous_weights[i], f"PositiveChange_{i}")
            model.addConstr(neg_change[i] >= previous_weights[i] - w[i], f"NegativeChange_{i}")
        
        # Total turnover constraint
        model.addConstr(
            quicksum(pos_change[i] + neg_change[i] for i in range(N)) <= 2 * max_turnover,
            "TurnoverLimit"
        )
        
        # Add transaction costs to the objective
        transaction_cost_term = quicksum(
            (pos_change[i] + neg_change[i]) * bond_data.iloc[i]['BidAskSpread'] 
            for i in range(N)
        )
    else:
        transaction_cost_term = 0

    # Optimize the model
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        #print("Optimal solution found. List of all weights:")
        weights = [w[i].X for i in range(N)]  # Get the optimized weights for bonds
        return np.array(weights)

    if model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
        return None



# 

0        2023-01-01
1        2023-01-01
2        2023-01-01
3        2023-01-01
4        2023-01-01
            ...    
36495    2023-12-31
36496    2023-12-31
36497    2023-12-31
36498    2023-12-31
36499    2023-12-31
Name: Date, Length: 36500, dtype: object
0       2023-01-01
1       2023-01-01
2       2023-01-01
3       2023-01-01
4       2023-01-01
           ...    
36495   2023-12-31
36496   2023-12-31
36497   2023-12-31
36498   2023-12-31
36499   2023-12-31
Name: Date, Length: 36500, dtype: datetime64[ns]
